In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score, sensitivity_score, specificity_score
from xgboost import XGBClassifier
from shaphypetune import BoostRFE
from sklearn.base import BaseEstimator
from sklearn.utils.metaestimators import if_delegate_has_method

import torch
import re, pickle, random, os
import warnings
warnings.filterwarnings('ignore')

from collections import Counter

In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

RANDOM_STATE = 42
seed_everything(seed=RANDOM_STATE)

In [3]:
col_names = ['class','T3-resin uptake','total serum thyroxin','total serum triiodotyronine','basal thyroid-stimulating hormone(TSH)','max difference of TSH']
df = pd.read_csv('new-thyroid.csv', names = col_names)
df.shape

(215, 6)

In [4]:
df.head()

,class,T3-resin uptake,total serum thyroxin,total serum triiodotyronine,basal thyroid-stimulating hormone(TSH),max difference of TSH
0,1,107,10.1,2.2,0.9,2.7
1,1,113,9.9,3.1,2.0,5.9
2,1,127,12.9,2.4,1.4,0.6
3,1,109,5.3,1.6,1.4,1.5
4,1,105,7.3,1.5,1.5,-0.1


In [5]:
df['class'].value_counts()

1    150
2     35
3     30
Name: class, dtype: int64

In [6]:
df.isnull().sum()

class                                     0
T3-resin uptake                           0
total serum thyroxin                      0
total serum triiodotyronine               0
basal thyroid-stimulating hormone(TSH)    0
max difference of TSH                     0
dtype: int64

In [7]:
df.describe()

,class,T3-resin uptake,total serum thyroxin,total serum triiodotyronine,basal thyroid-stimulating hormone(TSH),max difference of TSH
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000
mean,1.441860,109.595349,9.804651,2.050233,2.880000,4.199070
std,0.726737,13.145447,4.697362,1.419486,6.118031,8.070519
min,1.000000,65.000000,0.500000,0.200000,0.100000,-0.700000
25%,1.000000,103.000000,7.100000,1.350000,1.000000,0.550000
50%,1.000000,110.000000,9.200000,1.700000,1.300000,2.000000
75%,2.000000,117.500000,11.300000,2.200000,1.700000,4.100000
max,3.000000,144.000000,25.300000,10.000000,56.400000,56.300000


# Train and Test Split

In [8]:
x = df.drop(['class'], axis=1)
y = df['class']

X_train,X_test,y_train,y_test=train_test_split(x, y, train_size=0.8, stratify = y, random_state=100)

In [9]:
y_train.shape, y_test.shape

((172,), (43,))

In [10]:
y_train.value_counts()/len(y_train)

1    0.697674
2    0.162791
3    0.139535
Name: class, dtype: float64

In [11]:
y_test.value_counts()/len(y_test)

1    0.697674
2    0.162791
3    0.139535
Name: class, dtype: float64

In [12]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

# Model Building

In [13]:
model = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()
balanced_acc = list()
GMean = list()
sensi = list()
speci = list()

In [14]:
def test_eval(clf_model, X_test, y_test, algo=None):

    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('='*60)
    print('AUC-ROC')
    print(roc_auc_score(y_test, y_prob, multi_class='ovr'), "\n")
    print('Balanced Accuracy')
    print(balanced_accuracy_score(y_test, y_pred),"\n")
    print('Geometric Mean Score')
    print(geometric_mean_score(y_test, y_pred, average='macro'),"\n")
    print('Sensitivity')
    print(sensitivity_score(y_test, y_pred, average='macro'),"\n")
    print('Specificity')
    print(specificity_score(y_test, y_pred, average='macro'))
          
    model.append(algo)
    precision.append(precision_score(y_test,y_pred, average='macro'))
    recall.append(recall_score(y_test,y_pred, average='macro'))
    F1score.append(f1_score(y_test,y_pred, average='macro'))
    AUCROC.append(roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro'))
    balanced_acc.append(balanced_accuracy_score(y_test, y_pred))
    GMean.append(geometric_mean_score(y_test, y_pred, average='macro'))
    sensi.append(sensitivity_score(y_test, y_pred, average='macro'))
    speci.append(specificity_score(y_test, y_pred, average='macro'))


## Model-1: Logistic Regression

In [15]:
clf_LR = LogisticRegression(multi_class='ovr', solver='liblinear')
clf_LR.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [16]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression')

Confusion Matrix
[[30  0  0]
 [ 2  5  0]
 [ 1  0  5]] 

Classification Report
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        30
           2       1.00      0.71      0.83         7
           3       1.00      0.83      0.91         6

    accuracy                           0.93        43
   macro avg       0.97      0.85      0.90        43
weighted avg       0.94      0.93      0.93        43
 

AUC-ROC
0.9967890967890968 

Balanced Accuracy
0.8492063492063493 

Geometric Mean Score
0.8853715513177414 

Sensitivity
0.8492063492063493 

Specificity
0.923076923076923


## Model-2: Decision Tree

In [17]:
clf_DT = DecisionTreeClassifier()
clf_DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [18]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree')

Confusion Matrix
[[29  0  1]
 [ 2  5  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.94      0.97      0.95        30
           2       1.00      0.71      0.83         7
           3       0.86      1.00      0.92         6

    accuracy                           0.93        43
   macro avg       0.93      0.89      0.90        43
weighted avg       0.94      0.93      0.93        43
 

AUC-ROC
0.9166798666798667 

Balanced Accuracy
0.8936507936507937 

Geometric Mean Score
0.9163905498048526 

Sensitivity
0.8936507936507937 

Specificity
0.9397089397089399


## Model-3: Gaussian NB

In [19]:
clf_NB = GaussianNB()
clf_NB.fit(X_train, y_train)

GaussianNB()

In [20]:
test_eval(clf_NB, X_test, y_test, 'Gaussian NB')

Confusion Matrix
[[29  0  1]
 [ 0  7  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       1.00      0.97      0.98        30
           2       1.00      1.00      1.00         7
           3       0.86      1.00      0.92         6

    accuracy                           0.98        43
   macro avg       0.95      0.99      0.97        43
weighted avg       0.98      0.98      0.98        43
 

AUC-ROC
0.9965811965811966 

Balanced Accuracy
0.9888888888888889 

Geometric Mean Score
0.9899393819724418 

Sensitivity
0.9888888888888889 

Specificity
0.990990990990991


## Model-4: K-Nearest Neighbour

In [21]:
clf_KNN = KNeighborsClassifier()
clf_KNN.fit(X_train, y_train)

KNeighborsClassifier()

In [22]:
test_eval(clf_KNN, X_test, y_test, 'KNN')

Confusion Matrix
[[30  0  0]
 [ 3  4  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        30
           2       1.00      0.57      0.73         7
           3       1.00      1.00      1.00         6

    accuracy                           0.93        43
   macro avg       0.97      0.86      0.89        43
weighted avg       0.94      0.93      0.92        43
 

AUC-ROC
0.9987179487179487 

Balanced Accuracy
0.8571428571428571 

Geometric Mean Score
0.8894991799933213 

Sensitivity
0.8571428571428571 

Specificity
0.923076923076923


## MODEL-5 Support Vector Classifier

In [23]:
clf_SVC = SVC(probability=True)
clf_SVC.fit(X_train, y_train)

SVC(probability=True)

In [24]:
test_eval(clf_SVC, X_test, y_test, 'SVC')

Confusion Matrix
[[29  0  1]
 [ 2  5  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.94      0.97      0.95        30
           2       1.00      0.71      0.83         7
           3       0.86      1.00      0.92         6

    accuracy                           0.93        43
   macro avg       0.93      0.89      0.90        43
weighted avg       0.94      0.93      0.93        43
 

AUC-ROC
0.9965811965811966 

Balanced Accuracy
0.8936507936507937 

Geometric Mean Score
0.9163905498048526 

Sensitivity
0.8936507936507937 

Specificity
0.9397089397089399


# MODEL-6 XGBoost

In [25]:
class BoostRFEWrap(BaseEstimator, BoostRFE):

    @if_delegate_has_method(delegate='estimator')
    def predict_proba(self, X):
        return self.predict(X, method='predict_proba')


xgb_params = {'max_depth': (5,10),
          'learning_rate': (0.001, 0.3),
          'n_estimators': (25, 200),
          'reg_alpha' : (0.1, 1),
          'reg_lambda': (0.1, 1),
          'subsample': (0.5,  0.9),
          'colsample_bytree': (0.5,1),
          'min_child_weight': (0,10),
          'gamma': (0,1)}

clf_XGB = BoostRFEWrap(XGBClassifier(), param_grid=xgb_params, importance_type='shap_importances', train_importance=False, min_features_to_select=1, step=1)

In [26]:
xclf = clf_XGB.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=6, verbose=0)


512 trials detected for ('max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 'subsample', 'colsample_bytree', 'min_child_weight', 'gamma')

trial: 0001 ### iterations: 00004 ### eval_score: 0.04651
trial: 0002 ### iterations: 00007 ### eval_score: 0.04651
trial: 0003 ### iterations: 00003 ### eval_score: 0.02326
trial: 0004 ### iterations: 00005 ### eval_score: 0.06977
trial: 0005 ### iterations: 00005 ### eval_score: 0.04651
trial: 0006 ### iterations: 00004 ### eval_score: 0.02326
trial: 0007 ### iterations: 00003 ### eval_score: 0.02326
trial: 0008 ### iterations: 00003 ### eval_score: 0.02326
trial: 0009 ### iterations: 00002 ### eval_score: 0.04651
trial: 0010 ### iterations: 00002 ### eval_score: 0.04651
trial: 0011 ### iterations: 00003 ### eval_score: 0.04651
trial: 0012 ### iterations: 00008 ### eval_score: 0.04651
trial: 0013 ### iterations: 00002 ### eval_score: 0.02326
trial: 0014 ### iterations: 00006 ### eval_score: 0.0
trial: 0015 ### iterations: 00

trial: 0140 ### iterations: 00003 ### eval_score: 0.06977
trial: 0141 ### iterations: 00001 ### eval_score: 0.02326
trial: 0142 ### iterations: 00001 ### eval_score: 0.02326
trial: 0143 ### iterations: 00000 ### eval_score: 0.09302
trial: 0144 ### iterations: 00000 ### eval_score: 0.09302
trial: 0145 ### iterations: 00002 ### eval_score: 0.04651
trial: 0146 ### iterations: 00001 ### eval_score: 0.09302
trial: 0147 ### iterations: 00001 ### eval_score: 0.25581
trial: 0148 ### iterations: 00001 ### eval_score: 0.25581
trial: 0149 ### iterations: 00003 ### eval_score: 0.06977
trial: 0150 ### iterations: 00003 ### eval_score: 0.06977
trial: 0151 ### iterations: 00010 ### eval_score: 0.04651
trial: 0152 ### iterations: 00010 ### eval_score: 0.04651
trial: 0153 ### iterations: 00001 ### eval_score: 0.04651
trial: 0154 ### iterations: 00004 ### eval_score: 0.04651
trial: 0155 ### iterations: 00003 ### eval_score: 0.06977
trial: 0156 ### iterations: 00004 ### eval_score: 0.09302
trial: 0157 ##

trial: 0281 ### iterations: 00001 ### eval_score: 0.04651
trial: 0282 ### iterations: 00011 ### eval_score: 0.02326
trial: 0283 ### iterations: 00006 ### eval_score: 0.04651
trial: 0284 ### iterations: 00008 ### eval_score: 0.04651
trial: 0285 ### iterations: 00006 ### eval_score: 0.02326
trial: 0286 ### iterations: 00006 ### eval_score: 0.02326
trial: 0287 ### iterations: 00006 ### eval_score: 0.04651
trial: 0288 ### iterations: 00006 ### eval_score: 0.04651
trial: 0289 ### iterations: 00002 ### eval_score: 0.04651
trial: 0290 ### iterations: 00012 ### eval_score: 0.02326
trial: 0291 ### iterations: 00005 ### eval_score: 0.04651
trial: 0292 ### iterations: 00005 ### eval_score: 0.04651
trial: 0293 ### iterations: 00011 ### eval_score: 0.02326
trial: 0294 ### iterations: 00006 ### eval_score: 0.04651
trial: 0295 ### iterations: 00003 ### eval_score: 0.02326
trial: 0296 ### iterations: 00003 ### eval_score: 0.02326
trial: 0297 ### iterations: 00005 ### eval_score: 0.04651
trial: 0298 ##

trial: 0423 ### iterations: 00010 ### eval_score: 0.04651
trial: 0424 ### iterations: 00010 ### eval_score: 0.04651
trial: 0425 ### iterations: 00006 ### eval_score: 0.04651
trial: 0426 ### iterations: 00001 ### eval_score: 0.06977
trial: 0427 ### iterations: 00004 ### eval_score: 0.09302
trial: 0428 ### iterations: 00004 ### eval_score: 0.09302
trial: 0429 ### iterations: 00002 ### eval_score: 0.02326
trial: 0430 ### iterations: 00002 ### eval_score: 0.04651
trial: 0431 ### iterations: 00000 ### eval_score: 0.09302
trial: 0432 ### iterations: 00000 ### eval_score: 0.09302
trial: 0433 ### iterations: 00002 ### eval_score: 0.06977
trial: 0434 ### iterations: 00002 ### eval_score: 0.09302
trial: 0435 ### iterations: 00002 ### eval_score: 0.25581
trial: 0436 ### iterations: 00008 ### eval_score: 0.18605
trial: 0437 ### iterations: 00000 ### eval_score: 0.09302
trial: 0438 ### iterations: 00001 ### eval_score: 0.09302
trial: 0439 ### iterations: 00010 ### eval_score: 0.04651
trial: 0440 ##

In [27]:
test_eval(xclf, X_test, y_test, 'XGB')

Confusion Matrix
[[30  0  0]
 [ 0  7  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        30
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         6

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43
 

AUC-ROC
1.0 

Balanced Accuracy
1.0 

Geometric Mean Score
1.0 

Sensitivity
1.0 

Specificity
1.0


# MODEL-7 Stacking

In [28]:
clf_DT = DecisionTreeClassifier()
clf_SVC = SVC(probability=True)
clf_KNN = KNeighborsClassifier()
clf_LR = LogisticRegression()

sclf = StackingClassifier(classifiers=[clf_DT, clf_SVC, clf_KNN], use_probas=True, average_probas=False, meta_classifier=clf_LR)

In [29]:
sclf.fit(X_train, y_train)
test_eval(sclf, X_test, y_test, 'Stacking')

Confusion Matrix
[[29  0  1]
 [ 2  5  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.94      0.97      0.95        30
           2       1.00      0.71      0.83         7
           3       0.86      1.00      0.92         6

    accuracy                           0.93        43
   macro avg       0.93      0.89      0.90        43
weighted avg       0.94      0.93      0.93        43
 

AUC-ROC
0.9965811965811966 

Balanced Accuracy
0.8936507936507937 

Geometric Mean Score
0.9163905498048526 

Sensitivity
0.8936507936507937 

Specificity
0.9397089397089399


# MODEL-8 Bagging

In [30]:
clf_bg = BaggingClassifier(clf_DT)

In [31]:
clf_bg.fit(X_train, y_train)
test_eval(clf_bg, X_test, y_test, 'Bagging')

Confusion Matrix
[[30  0  0]
 [ 3  4  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        30
           2       1.00      0.57      0.73         7
           3       1.00      1.00      1.00         6

    accuracy                           0.93        43
   macro avg       0.97      0.86      0.89        43
weighted avg       0.94      0.93      0.92        43
 

AUC-ROC
0.9991452991452991 

Balanced Accuracy
0.8571428571428571 

Geometric Mean Score
0.8894991799933213 

Sensitivity
0.8571428571428571 

Specificity
0.923076923076923


In [32]:
clf_eval_df = pd.DataFrame({'model':model,
                            'precision':precision,
                            'recall':recall,
                            'f1-score':F1score,
                            'AUC-ROC':AUCROC,
                            'balanced_acc':balanced_acc,
                            'GMean':GMean,
                            'sensitivity':sensi,
                            'specificity':speci})
clf_eval_df

,model,precision,recall,f1-score,AUC-ROC,balanced_acc,GMean,sensitivity,specificity
0,Logistic Regression,0.969697,0.849206,0.898268,0.996789,0.849206,0.885372,0.849206,0.923077
1,Decision Tree,0.930876,0.893651,0.902410,0.916680,0.893651,0.916391,0.893651,0.939709
2,Gaussian NB,0.952381,0.988889,0.968709,0.996581,0.988889,0.989939,0.988889,0.990991
3,KNN,0.969697,0.857143,0.893218,0.998718,0.857143,0.889499,0.857143,0.923077
4,SVC,0.930876,0.893651,0.902410,0.996581,0.893651,0.916391,0.893651,0.939709
5,XGB,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,Stacking,0.930876,0.893651,0.902410,0.996581,0.893651,0.916391,0.893651,0.939709
7,Bagging,0.969697,0.857143,0.893218,0.999145,0.857143,0.889499,0.857143,0.923077


In [33]:
clf_eval_df.to_excel("D:\Skripsi\Final\project_newthyroid_final.xlsx")